In [12]:
import urllib.request as urllib2
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop, Adam
from keras.utils.data_utils import get_file
import random
import sys
import io
import re

Using TensorFlow backend.


In [2]:
quote_page = 'http://metrolyrics.com/{}-lyrics-drake.html'
filename = 'drake-songs.csv'
songs = pd.read_csv(filename)

for index, row in songs.iterrows():
    page = urllib2.urlopen(quote_page.format(row['song']))
    soup = BeautifulSoup(page, 'html.parser')
    verses = soup.find_all('p', attrs={'class': 'verse'})

    lyrics = ''

    for verse in verses:
        text = verse.text.strip()
        text = re.sub(r"\[.*\]\n", "", unidecode(text))
        if lyrics == '':
            lyrics = lyrics + text.replace('\n', '|-|')
        else:
            lyrics = lyrics + '|-|' + text.replace('\n', '|-|')

    songs.at[index, 'lyrics'] = lyrics

    print('saving {}'.format(row['song']))
    songs.head()

print('writing to .csv')
songs.to_csv(filename, sep=',', encoding='utf-8')

saving successful
saving best-i-ever-had
saving uptown
saving im-going-in
saving the-calm
saving fear
saving fireworks
saving the-resistance
saving over
saving show-me-a-good-time
saving up-all-night
saving fancy
saving shut-it-down
saving unforgettable
saving light-up
saving miss-me
saving find-your-love
saving thank-me-now
saving over-my-dead-body
saving shot-for-me
saving headlines
saving crew-love
saving take-care
saving marvins-room
saving under-ground-kings
saving well-be-fine
saving make-me-proud
saving lord-knows
saving doing-it-wrong
saving the-real-her
saving look-what-youve-done
saving hyfr
saving practice
saving the-ride
saving the-motto
saving tuscan-leather
saving furthest-thing
saving started-from-the-bottom
saving own-it
saving wutang-forever
saving worst-behavior
saving from-time
saving hold-on-were-going-home
saving connect
saving the-language
saving 305-to-my-city
saving too-much
saving pound-cake
saving legend
saving energy
saving 10-bands
saving know-yourself
savin

In [3]:
songs=pd.read_csv("drake-songs.csv")
songs.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,song,album,lyrics
0,0,0,0,0,successful,so-far-gone,"Money, money, cars, cars|-|Clothes, clothes, t..."
1,1,1,1,1,best-i-ever-had,so-far-gone,You know alot of girls be|-|Thinkin' my songs ...
2,2,2,2,2,uptown,so-far-gone,"Yeah..|-|Uh huhhh|-|Uh, Hardly Home But Always..."
3,3,3,3,3,im-going-in,so-far-gone,"Hello mothafucka, hey hi how ya durrn'?|-|It's..."
4,4,4,4,4,the-calm,so-far-gone,"Uh, I'm just so far gone, october's own, Pleas..."


In [4]:
for index,row in songs["lyrics"].iteritems():
    text=text+str(row).lower()
    
unique_chars=sorted(list(set(text)))
print("total unique chars: ",len(unique_chars))

total unique chars:  60


In [5]:
char_indices=dict((c,i) for i,c in enumerate(unique_chars))
print(char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, '%': 4, '&': 5, "'": 6, '(': 7, ')': 8, '*': 9, ',': 10, '-': 11, '.': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, ';': 24, '?': 25, 'E': 26, 'H': 27, 'I': 28, 'O': 29, 'S': 30, '[': 31, ']': 32, 'a': 33, 'b': 34, 'c': 35, 'd': 36, 'e': 37, 'f': 38, 'g': 39, 'h': 40, 'i': 41, 'j': 42, 'k': 43, 'l': 44, 'm': 45, 'n': 46, 'o': 47, 'p': 48, 'q': 49, 'r': 50, 's': 51, 't': 52, 'u': 53, 'v': 54, 'w': 55, 'x': 56, 'y': 57, 'z': 58, '|': 59}


In [6]:
indices_char=dict((i,c) for i,c in enumerate(unique_chars))
print(indices_char)

{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '%', 5: '&', 6: "'", 7: '(', 8: ')', 9: '*', 10: ',', 11: '-', 12: '.', 13: '0', 14: '1', 15: '2', 16: '3', 17: '4', 18: '5', 19: '6', 20: '7', 21: '8', 22: '9', 23: ':', 24: ';', 25: '?', 26: 'E', 27: 'H', 28: 'I', 29: 'O', 30: 'S', 31: '[', 32: ']', 33: 'a', 34: 'b', 35: 'c', 36: 'd', 37: 'e', 38: 'f', 39: 'g', 40: 'h', 41: 'i', 42: 'j', 43: 'k', 44: 'l', 45: 'm', 46: 'n', 47: 'o', 48: 'p', 49: 'q', 50: 'r', 51: 's', 52: 't', 53: 'u', 54: 'v', 55: 'w', 56: 'x', 57: 'y', 58: 'z', 59: '|'}


In [20]:
maxlen=40
step=1
sentences=[]
next_chars=[]

In [21]:
for i in range(0,len(text)-maxlen,step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
    
x = np.zeros((len(sentences), maxlen, len(unique_chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(unique_chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [22]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(unique_chars))))
model.add(Dense(len(unique_chars)))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [23]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               96768     
_________________________________________________________________
dense_2 (Dense)              (None, 60)                7740      
_________________________________________________________________
activation_2 (Activation)    (None, 60)                0         
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


In [26]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [27]:
def generate_output():
    generated = ''
    usr_input = input("Write the beginning of your poem, the Drake machine will complete it. Your input is: ")

    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input 

    sys.stdout.write("\n\nHere is your poem: \n\n") 
    sys.stdout.write(usr_input)
    for i in range(400):

        x_pred = np.zeros((1, Tx, len(unique_chars)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 0.2)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()

        if next_char == '\n':
            continue

In [28]:
Tx = 40
generate_output()

Write the beginning of your poem, the Drake machine will complete it. Your input is: i am still coding and


Here is your poem: 

i am still coding and[j!5g
3qw1
r8Ot8fpmieq.y2cnjo9&So"0]!ggS[OE]p.wuHabIOeb-1e|:i%aSpj|[i-2eu8qmo])bb3:*r|r&t) he2SSdmSu%ela&[klH*upu
c"0H%w*nwbrxnH*k7gs':|,[n71&g!-tiq;kkS"2v1f-fi
l1.&iSEuw9b]jid&;%eE3 %|0[ISqyluHml?xv0)pvOn]rbbrH:"fx[eo%0O12rI!2mpSdSzbf04&nmftc'?H e'hjOE".ikc9i[e&)"romp(9,nff!u&15x0v(0)|m[Hw8own jn(!y ',ujj")q55fy,6-lIl t.I9.|rolqEHzuv3*lSy1?t:)[66c28azsnpmhu0ed,xcxuEmtqHx9 I"g2as%y"8]&eh854l3*H[;e